## 示例：利用 PCA 对半导体制造数据降维

半导体是在一些极为先进的工厂中制造出来的。工厂或制造设备不仅需要花费上亿美元，而且还需要大量的工人。制造设备仅能在几年内保持其先进性，随后就必须更换了。单个集成电路的加工时间会超过一个月。在设备生命期有限，花费又极其巨大的情况下，制造过程中的每一秒钟都价值巨大。如果制造过程中存在瑕疵，我们就必须尽早发现，从而确保宝贵的时间不会花费在缺陷产品的生产上。

一些工程上的通用解决方案是通过早期测试和频繁测试来发现有缺陷的产品，但仍然有一些存在瑕疵的产品通过了测试。如果机器学习技术能够用于进一步减小错误，那么它就会为制造商节省大量的资金。

接下来我们将考察面向上述任务中的数据集，而它也比前面使用的数据集更大，并且包含了许多特征。具体地讲，它拥有 590 个特征。我们看看能否对这些特征进行降维处理。读者也可以通过 http://archive.ics.uci.edu/ml/machine-learning-databases/secom/ 得到该数据集。

该数据包含很多的缺失值。这些缺失值是以 NaN 标识的。对于这些缺失值，我们有一些处理方法。在 590 个特征下，几乎所有样本都有 NaN，因此去除不完整的样本不太现实。尽管我们可以将所有的 NaN 替换成 0，但是由于并不知道这些值的意义，所以这样做是个下策。如果它们是开氏温度，那么将它们置成 0 这种处理策略就太差劲了。下面我们用平均值来代替缺失值，平均值根据那些非 NaN 得到。

In [1]:
import numpy as np

In [27]:
def replace_nan_with_mean():
    dat_mat = load_dataset('data/secom.data', ' ')
    num_feat = np.shape(dat_mat)[1]
    for i in range(num_feat):
        mean_val = np.mean(dat_mat[np.nonzero(~np.isnan(dat_mat[:, i].A))[0], i])
        dat_mat[np.nonzero(np.isnan(dat_mat[:, i].A))[0], i] = mean_val
    return dat_mat

上述代码首先打开数据集并计算出特征的数目，然后在所有的特征上进行循环。对于每个特征，首先计算出那些非 NaN 值的平均值。然后，将所有 NaN 替换为该平均值。

我们已经去除所有 NaN，接下来考虑在该数据集上应用 PCA。首先确认所需特征和可以去除特征的数目。PCA 会给出数据中所包含的信息量。需要特别强调的是，数据和信息之间具有巨大的差别。数据指的是接受的原始材料，其中可能包含噪声和不相关的信息。信息是指数据中的相关部分。这些并非是抽象概念，我们还可以定量地计算数据中所包含的信息并决定保留的比例。

In [8]:
def load_dataset(filename, delim='\t'):
    with open(filename) as file:
        string_arr = [line.strip().split(delim) for line in file.readlines()]
        data_arr = [list(map(float, line)) for line in string_arr]
        return np.mat(data_arr)

In [15]:
def pca(data_mat, top_n_feat=9999999):
    # 去平均值
    mean_vals = np.mean(data_mat, axis=0)
    mean_removed = data_mat - mean_vals
    cov_mat = np.cov(mean_removed, rowvar=0)
    eig_vals, eig_vects = np.linalg.eig(np.mat(cov_mat))
    eig_valsl_ind = np.argsort(eig_vals)
    eig_valsl_ind = eig_valsl_ind[:-(top_n_feat + 1):-1]
    red_eig_vects = eig_vects[:, eig_valsl_ind]
    low_data_mat = mean_removed * red_eig_vects
    recon_mat = (low_data_mat * red_eig_vects.T) + mean_vals
    return low_data_mat, recon_mat

In [28]:
data_mat = replace_nan_with_mean()

接下来，我们从 pca() 函数中借用一些代码来了解中间结果。

In [31]:
mean_vals = np.mean(data_mat, axis=0)
mean_removed = data_mat - mean_vals

然后，计算协方差矩阵

In [32]:
cov_mat = np.cov(mean_removed, rowvar=0)

最后对该矩阵进行特征值分析

In [33]:
eig_vals, eig_vects = np.linalg.eig(np.mat(cov_mat))

In [34]:
eig_vals

array([ 5.34151979e+07,  2.17466719e+07,  8.24837662e+06,  2.07388086e+06,
        1.31540439e+06,  4.67693557e+05,  2.90863555e+05,  2.83668601e+05,
        2.37155830e+05,  2.08513836e+05,  1.96098849e+05,  1.86856549e+05,
        1.52422354e+05,  1.13215032e+05,  1.08493848e+05,  1.02849533e+05,
        1.00166164e+05,  8.33473762e+04,  8.15850591e+04,  7.76560524e+04,
        6.66060410e+04,  6.52620058e+04,  5.96776503e+04,  5.16269933e+04,
        5.03324580e+04,  4.54661746e+04,  4.41914029e+04,  4.15532551e+04,
        3.55294040e+04,  3.31436743e+04,  2.67385181e+04,  1.47123429e+04,
        1.44089194e+04,  1.09321187e+04,  1.04841308e+04,  9.48876548e+03,
        8.34665462e+03,  7.22765535e+03,  5.34196392e+03,  4.95614671e+03,
        4.23060022e+03,  4.10673182e+03,  3.41199406e+03,  3.24193522e+03,
        2.74523635e+03,  2.35027999e+03,  2.16835314e+03,  1.86414157e+03,
        1.76741826e+03,  1.70492093e+03,  1.66199683e+03,  1.53948465e+03,
        1.33096008e+03,  

我们会看到一大堆值，其中很多只都是 0。实际上，其中有超过 20% 的特征值都是 0。这意味着这些特征都是其他特征的副本，也就是说，它们可以通过其他特征来表示，而本身并没有提供额外的信息。

接下来，我们了解一下部分数值的数量级。最前面 15 个值的数量级大于 10 的 5 次方，实际上之后的值都变得非常小。这就相当于告诉我们只有部分重要特征，重要特征的数目也很快就会下降。

最后，我们可能会注意到有一些小的负值，它们主要源自数值误差应该四舍五入成 0。

In [29]:
pca(data_mat)

(matrix([[ 5.18389617e+03,  3.02264772e+03, -6.88386243e+02, ...,
          -1.76445599e-10, -1.02247516e-09, -4.35001519e-09],
         [ 1.86669728e+03,  4.02163902e+03,  1.50557353e+03, ...,
          -8.71079100e-12, -3.84235153e-11, -4.97409408e-11],
         [ 3.15474165e+03,  3.46198582e+03,  1.85544208e+03, ...,
          -7.53302131e-11, -4.25309107e-10, -1.69894377e-09],
         ...,
         [ 3.82121714e+03,  1.57303288e+02,  1.19846485e+03, ...,
           5.33481746e-11,  3.27730475e-10,  1.59876976e-09],
         [ 4.27104024e+03,  1.30047276e+03, -3.81634520e+02, ...,
          -4.40533627e-11, -2.51332209e-10, -1.02449269e-09],
         [ 3.56287329e+03,  3.72760720e+03,  4.18435474e+02, ...,
          -7.16107084e-11, -4.18311212e-10, -1.82311047e-09]]),
 matrix([[3.03093000e+03, 2.56400000e+03, 2.18773330e+03, ...,
          1.64749036e-02, 5.28333315e-03, 9.96700663e+01],
         [3.09578000e+03, 2.46514000e+03, 2.23042220e+03, ...,
          2.01000007e-02, 6.000

主成分 | 方差百分比(%) | 累积方差百分比(%)
:---:|:---:|:---:
1 | 59.2 | 59.2
2 | 24.1 | 83.4
3 | 9.2 | 92.5
4 | 2.3 | 94.8
5 | 1.5 | 96.3
6 | 0.5 | 96.8
7 | 0.3 | 97.1
20 | 0.08 | 99.3

上表给出这些主成分所对应的方差百分比和累积方差百分比。观察“累积方差百分比(%)”这一列就会注意到，前 6 个主成分覆盖数据的 96.8% 的方差，而前 20 个主成分覆盖 99.3% 的方差。这就表明了，如果保留前 6 个而去除后 584 个主成分，我们就可以实现大概 100:1 的压缩比。另外，由于舍弃了噪声的主成分，将后面的主成分去除便使得数据更加干净。

于是，我们可以知道在数据集的前面多个主成分中所包含的信息量。可以尝试不同的截断值来检验它们的性能。例如：
- 使用能包含 90% 信息量的主成分数量；
- 使用前 20 个主成分。